In [ ]:
import os
import sys
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
from numpy import linalg as la
from scipy import linalg
from scipy.spatial.distance import directed_hausdorff

sys.path.insert(0, '../../../methods')
from capon import *

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []
hausdorff_list1 = []
hausdorff_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data/experiment_2/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['cm'] for s in data]

    # truths = np.stack(truths)
    # data = np.stack(data)

    # apply capon beamformer
    res = 1
    p_list = []
    r_list = []
    h_list = []

    for i in range(len(data)):
        R = data[i]
        M = R.shape[0]
        N = truths[i].shape[0]
        truths[i] = truths[i].reshape(-1,)

        preds, spectrum = capon(R,M,N,sensor_pos[i],res)

        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])
        # truths[i], preds = match_closest_pairs(truths[i], preds)
        r_list.append((truths[i]-preds)**2)
        p_list.append(preds)

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))
        
        if snr==10 and i==0:
             np.save(f'../../../results/experiment_2/scenario_1/capon_spectrum_{truths[0]}deg.npy', spectrum)

    rmse = np.sqrt(np.mean(np.concatenate(r_list)))
    hausdorff = np.mean(h_list)
    rmse_list1.append(rmse)
    hausdorff_list1.append(hausdorff)
    
    print(f"snr {snr}dB, T {1000}, test-rmse {rmse:.4f}, test-hausdorff {hausdorff:.4f}")
    
    with open(f'../../../results/experiment_2/scenario_1/capon_preds_snr{snr}_t1000', "wb") as fp: 
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_2/scenario_1/capon_truths_snr{snr}_t1000', "wb") as fp: 
        pickle.dump(truths, fp)
    
for T in T_list:
    # load data
    data = np.load(f'../../../data/experiment_2/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['cm'] for s in data]

    # truths = np.stack(truths)
    # data = np.stack(data)

    # apply capon beamformer
    res = 1
    p_list = []
    r_list = []
    h_list = []

    for i in range(len(data)):
        R = data[i]
        M = R.shape[0]
        N = truths[i].shape[0]
        truths[i] = truths[i].reshape(-1,)

        preds, spectrum = capon(R,M,N,sensor_pos[i],res)

        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])
        # truths[i], preds = match_closest_pairs(truths[i], preds)
        r_list.append((truths[i]-preds)**2)
        p_list.append(preds)

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))

    rmse = np.sqrt(np.mean(np.concatenate(r_list)))
    hausdorff = np.mean(h_list)
    rmse_list2.append(rmse)
    hausdorff_list2.append(hausdorff)
    
    print(f"snr -10dB, T {T}, test-rmse {rmse:.4f}, test-hausdorff {hausdorff:.4f}")
    
    with open(f'../../../results/experiment_2/scenario_1/capon_preds_snr-10_t{T}', "wb") as fp: 
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_2/scenario_1/capon_truths_snr-10_t{T}', "wb") as fp: 
        pickle.dump(truths, fp)
    
np.save('../../../results/experiment_2/scenario_1/capon_rmse1.npy', rmse_list1)
np.save('../../../results/experiment_2/scenario_1/capon_rmse2.npy', rmse_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, rmse_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['capon'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, rmse_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['capon'])
plt.yscale("log")
plt.grid()